In [3]:
import numpy as np
import cupy as cp
import time

def matmul_cpu(A, B):
    rows_A, cols_A = A.shape
    rows_B, cols_B = B.shape
    assert cols_A == rows_B, "Количество столбцов A должно совпадать с количеством строк B"
    
    # Инициализация результирующей матрицы
    C = np.zeros((rows_A, cols_B))
    
    # Плоский цикл для вычисления индексов
    total_elements = rows_A * cols_B
    for idx in range(total_elements):
        i = idx // cols_B  # Номер строки результирующей матрицы
        j = idx % cols_B   # Номер столбца результирующей матрицы
        C[i, j] = sum(A[i, k] * B[k, j] for k in range(cols_A))
    
    return C

def matmul_gpu(A, B):
    A_gpu = cp.asarray(A)
    B_gpu = cp.asarray(B)
    
    rows_A, cols_A = A_gpu.shape
    rows_B, cols_B = B_gpu.shape
    assert cols_A == rows_B, "Количество столбцов A должно совпадать с количеством строк B"
    
    # Инициализация результирующей матрицы
    C_gpu = cp.zeros((rows_A, cols_B))
    
    # Плоский цикл для вычисления индексов
    total_elements = rows_A * cols_B
    for idx in range(total_elements):
        i = idx // cols_B  # Номер строки результирующей матрицы
        j = idx % cols_B   # Номер столбца результирующей матрицы
        C_gpu[i, j] = cp.sum(A_gpu[i, :] * B_gpu[:, j])
    
    return cp.asnumpy(C_gpu)

def validate_results(C_cpu, C_gpu):
    return np.allclose(C_cpu, C_gpu, atol=1e-6)

def measure_time(func, *args):
    start_time = time.time()
    result = func(*args)
    end_time = time.time()
    return result, end_time - start_time

def run_experiments(sizes):
    results = []
    for size in sizes:
        A = np.random.rand(size, size)
        B = np.random.rand(size, size)
        
        C_cpu, time_cpu = measure_time(matmul_cpu, A, B)
        C_gpu, time_gpu = measure_time(matmul_gpu, A, B)
        
        assert validate_results(C_cpu, C_gpu), f"Validation failed for size {size}"
        
        speedup = time_cpu / time_gpu
        results.append((size, time_cpu, time_gpu, speedup))
        print(f"Size: {size}x{size}, CPU Time: {time_cpu:.4f}s, GPU Time: {time_gpu:.4f}s, Speedup: {speedup:.2f}x")
    
    return results


In [4]:
# Размеры матриц для экспериментов
sizes = [100, 500, 1000, 1500, 2000]
results = run_experiments(sizes)

# Сохранение результатов в файл
import csv

with open('results0.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Size", "CPU Time (s)", "GPU Time (s)", "Speedup"])
    writer.writerows(results)

Size: 100x100, CPU Time: 0.1754s, GPU Time: 0.9527s, Speedup: 0.18x
Size: 500x500, CPU Time: 22.6590s, GPU Time: 10.9913s, Speedup: 2.06x
Size: 1000x1000, CPU Time: 183.8639s, GPU Time: 44.8291s, Speedup: 4.10x
Size: 1500x1500, CPU Time: 631.4437s, GPU Time: 108.7515s, Speedup: 5.81x
Size: 2000x2000, CPU Time: 2496.3273s, GPU Time: 185.6984s, Speedup: 13.44x
